# 1.Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

# Generate synthetic network traffic data
np.random.seed(42)
time_index = pd.date_range(start="2024-01-01", periods=1000, freq="H")

data = pd.DataFrame({
    "timestamp": time_index,
    "network_traffic": np.random.randint(50, 500, size=(1000,)),  # Random traffic load
    "latency": np.random.uniform(10, 100, size=(1000,))  # Random latency in ms
})

data.to_csv("./data/network_traffic.csv", index=False)
print("Sample data created successfully.")


Sample data created successfully.


C:\Users\KIIT\AppData\Local\Temp\ipykernel_5624\552226740.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start="2024-01-01", periods=1000, freq="H")


# 2.Model Building

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


## 2.1 LSTM

In [5]:


# Load dataset
data = pd.read_csv("./data/network_traffic.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])
data.set_index("timestamp", inplace=True)

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Prepare data for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled)
X_train, y_train = X[:-100], y[:-100]
X_test, y_test = X[-100:], y[-100:]

# Define LSTM model
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(50, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))
model.save("./models/network_traffic_lstm.h5")

print("Model training complete.")


Epoch 1/20


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.2107 - val_loss: 0.0788
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0841 - val_loss: 0.0787
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0832 - val_loss: 0.0794
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0842 - val_loss: 0.0781
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0895 - val_loss: 0.0781
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0843 - val_loss: 0.0782
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0831 - val_loss: 0.0800
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0864 - val_loss: 0.0788
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0877 - val_loss: 0.0773
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0838 - val_loss: 0.0773
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0837 - val_loss: 0.0806
Epoch 12/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0855 - val_loss: 0.0803

Model training complete.


## 2.2 Implement Bandwidth Optimization using Reinforcement Learning

In [ ]:
import gym
import numpy as np

class NetworkEnv(gym.Env):
    def __init__(self):
        self.state = np.random.randint(50, 500)  # Initial traffic load
        self.action_space = gym.spaces.Discrete(3)  # Increase, Decrease, Maintain
        self.observation_space = gym.spaces.Box(low=50, high=500, shape=(1,), dtype=np.int32)

    def step(self, action):
        if action == 0:
            self.state = max(50, self.state - 10)  # Reduce traffic load
        elif action == 1:
            self.state = min(500, self.state + 10)  # Increase traffic load

        reward = -abs(self.state - 250)  # Ideal load is 250
        done = False
        return np.array([self.state]), reward, done, {}

    def reset(self):
        self.state = np.random.randint(50, 500)
        return np.array([self.state])

env = NetworkEnv()


### 2.2.1 Train RL agent

In [ ]:
from stable_baselines3 import DQN

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("bandwidth_optimizer")
print("Reinforcement Learning Model Saved.")


## 2.3 Anomaly Detection (Autoencoder)

In [ ]:
from sklearn.ensemble import IsolationForest

# Train model
clf = IsolationForest(contamination=0.05)
clf.fit(data[["network_traffic"]])

# Predict anomalies
data["anomaly"] = clf.predict(data[["network_traffic"]])
data["anomaly"] = data["anomaly"].apply(lambda x: 1 if x == -1 else 0)
data.to_csv("network_traffic_anomalies.csv")
print("Anomaly detection completed.")


In [ ]:
# deploying API with FastAPI
from fastapi import FastAPI
import tensorflow as tf
import numpy as np

app = FastAPI()

# Load trained model
model = tf.keras.models.load_model("network_traffic_model.h5")

@app.get("/predict")
def predict(traffic: float, latency: float):
    input_data = np.array([[traffic, latency]])
    prediction = model.predict(input_data)
    return {"Predicted Traffic": float(prediction[0][0])}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)
